In [6]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from Modules.dataEngineering import dataEngineering
dataEngineering = dataEngineering()

Load ข้อมูล

In [7]:
df_ratings        = dataEngineering.loadRatings()['data']
df_ratings.shape

(2247560, 3)

learning_rate และ user_based ที่ดีที่สุด

In [8]:
from surprise import Dataset, Reader, KNNWithMeans
from surprise.model_selection import GridSearchCV

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_ratings[['UserID', 'MovieID', 'userRating']], reader)

In [8]:
sim_options = {
    'name': ['cosine', 'pearson'],
    'user_based': [False, True],
}
bsl_options = {
    'method': ['sgd'],
    'learning_rate': [0.0005, 0.005]
}
param_grid = {
    'k':[20, 60],
    'sim_options': sim_options,
    'bsl_options': bsl_options
}

# สร้างตัว GridSearchCV สำหรับการค้นหาพารามิเตอร์
gs = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing t

600m +

In [9]:
display(gs.best_score, gs.best_params)

{'rmse': 0.7749029916543432, 'mae': 0.5741506532172442}

{'rmse': {'k': 60,
  'sim_options': {'name': 'pearson', 'user_based': False},
  'bsl_options': {'method': 'sgd', 'learning_rate': 0.0005}},
 'mae': {'k': 20,
  'sim_options': {'name': 'pearson', 'user_based': False},
  'bsl_options': {'method': 'sgd', 'learning_rate': 0.0005}}}

In [10]:
options_KNNwithMeans = gs.best_params['rmse']
options_KNNwithMeans

{'k': 60,
 'sim_options': {'name': 'pearson', 'user_based': False},
 'bsl_options': {'method': 'sgd', 'learning_rate': 0.0005}}

In [10]:
from surprise.model_selection import train_test_split
from surprise import accuracy

In [11]:
train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)

model_KNNwithMeans = KNNWithMeans( 
    k=options_KNNwithMeans['k'],
    sim_options=options_KNNwithMeans['sim_options'], 
    bsl_options=options_KNNwithMeans['bsl_options']
)
model_KNNwithMeans.fit(train_set)

predictions = model_KNNwithMeans.test(test_set)
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.7664
MAE:  0.5682


In [12]:
def predict_model(pivot):
    matrix = pivot.copy()
    u_col = 0
    ratings = []
    for u_id, row in matrix.iterrows():
        for m_col, item in enumerate(row):
            m_id = matrix.columns[m_col]
            if pd.isnull(item):
                prep_rating = round(model_KNNwithMeans.predict(u_id, m_id).est, 4)
                ratings.append([u_id, m_id, prep_rating])
                matrix.iloc[u_col][m_id] = prep_rating
        u_col += 1
    
    recomment_unwatched = pd.DataFrame(ratings, columns=['UserID', 'MovieID', 'prep_Rating'])
    return matrix, recomment_unwatched.sort_values(by=['prep_Rating'], ascending=False)

In [13]:
df_pivot = df_ratings.pivot(index='UserID', columns='MovieID' , values='userRating')

In [14]:
mat, prep = predict_model(df_pivot.iloc[1:2])
mat

MovieID,1,2,3,4,5,6,7,8,9,10,...,188301,189203,189333,189713,192385,192389,192803,194448,195159,201773
UserID,,,,,,,,,,,,,,,,,,,,,
4,3.0,3.0684,2.7493,1.937,2.6409,4.0624,2.9131,2.6436,2.3832,3.1056,...,2.7912,3.7449,4.0,3.7049,3.5013,2.6821,3.1227,3.6345,5.0,3.2015


Save Models

In [15]:
import joblib

joblib.dump(model_KNNwithMeans, 'models/model_KNNwithMeans.pkl')

['models/model_KNNwithMeans.pkl']

Save Parameter

In [16]:
def human_format(num):
    num = float('{:.3g}'.format(num))
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    return '{}{}'.format('{:f}'.format(num).rstrip('0').rstrip('.'), ['', 'K', 'M', 'B', 'T'][magnitude])

In [17]:
import json
import os

folder = human_format(df_ratings.shape[0])

# Create directory if directory exists.
os.makedirs(f'./Parameter/{folder}/', exist_ok=True) 

with open(f'./Parameter/option_KNNwithMeans.json', 'w') as f:
    json.dump(options_KNNwithMeans, f, indent = 6)

with open(f'./Parameter/{folder}/option_KNNwithMeans.json', 'w') as f:
    json.dump(options_KNNwithMeans, f, indent = 6)